Import

In [458]:
import numpy as np
import os
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [459]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [460]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [461]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [462]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('./data/input')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS
image_id = 0

In [463]:
model_name = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
detection_model =  tf.saved_model.load('./model/TensorFlow_model_zoo/' + model_name + '/saved_model')

In [464]:
threshold = 0.4

In [465]:
detection_model.signatures['serving_default'].output_dtypes

{'raw_detection_boxes': tf.float32,
 'detection_anchor_indices': tf.float32,
 'detection_scores': tf.float32,
 'detection_boxes': tf.float32,
 'raw_detection_scores': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'detection_classes': tf.float32,
 'num_detections': tf.float32}

In [466]:
detection_model.signatures['serving_default'].output_shapes

{'raw_detection_boxes': TensorShape([1, 51150, 4]),
 'detection_anchor_indices': TensorShape([1, 100]),
 'detection_scores': TensorShape([1, 100]),
 'detection_boxes': TensorShape([1, 100, 4]),
 'raw_detection_scores': TensorShape([1, 51150, 91]),
 'detection_multiclass_scores': TensorShape([1, 100, 91]),
 'detection_classes': TensorShape([1, 100]),
 'num_detections': TensorShape([1])}

In [467]:
def run_inference_for_single_image(model, image, threshold):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed >0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [468]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np, threshold)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      min_score_thresh=threshold,
      line_thickness=4)

  # display(Image.fromarray(image_np))

In [469]:
def get_unnormalized_boxes(image_height, image_length, boxes):
    unnormalized_boxes = boxes.copy()
    unnormalized_boxes[ : , 0: : 2] = image_height * unnormalized_boxes[ : , 0: : 2]
    unnormalized_boxes[ : , 1: : 2] = image_length * unnormalized_boxes[ : , 1: : 2]
    return unnormalized_boxes

In [470]:

def clip_image_from_box(image_np, output_dict, threshold):
    #image_np
    im_height, im_length = image_np.shape[0], image_np.shape[1]
    boxes = get_unnormalized_boxes(im_height, im_length, output_dict['detection_boxes'])

    extend = 10
    #1: person; 3: car; 4: motorcycle; 6: bus; 8: truck
    detection_classes = (1, 3, 4)
    for i in range(boxes.shape[0]):
        if (output_dict['detection_classes'][i] in detection_classes 
             and output_dict['detection_scores'][i] > threshold):
             cropped_image = tf.image.crop_to_bounding_box(
                                                    image_np,
                                                    offset_height = int(max(boxes[i, 0] - extend, 0)),
                                                    offset_width = int(max(boxes[i, 1] - extend, 0)),
                                                    target_height = int(min(boxes[i, 2] - boxes[i, 0] +2 * extend, im_height - boxes[i, 0])),
                                                    target_width = int(min(boxes[i, 3] - boxes[i, 1] +2 * extend, im_length - boxes[i, 1]))
                                                    )
             cropped_image = np.array(cropped_image)
             #save_image(cropped_image, i)

             display(Image.fromarray(cropped_image))


In [471]:
def clip_image_from_merged_boxes(image_np, boxes):
    im_height, im_length = image_np.shape[0], image_np.shape[1]
    extend = 10
    for i in range(boxes.shape[0]):
        cropped_image = tf.image.crop_to_bounding_box(
                                                    image_np,
                                                    offset_height = int(max(boxes[i, 0] - extend, 0)),
                                                    offset_width = int(max(boxes[i, 1] - extend, 0)),
                                                    target_height = int(min(boxes[i, 2] - boxes[i, 0] +2 * extend, im_height - boxes[i, 0])),
                                                    target_width = int(min(boxes[i, 3] - boxes[i, 1] +2 * extend, im_length - boxes[i, 1]))
                                                    )
        cropped_image = np.array(cropped_image)
        save_image(cropped_image, i)

        # display(Image.fromarray(cropped_image))


In [472]:

def is_small_box(loc, w, h, thre=0.1):
    return ((loc[3]-loc[1])<thre*h)&((loc[2]-loc[0])<thre*w)


def merge_bound_box_near(output_dict, w, h, thre=0.1):
    '''
    merge bound boxes
    Args:
        locs-->list: [[x_min1,y_min1,x_max2,y_max2],...,[x_min1,y_min1,x_max2,y_max2]]
        w: the width of the image
        h: the height of the image
        thre: threshold for judging if two bound box is near enough
    '''
    locs = []
    boxes = get_unnormalized_boxes(w, h, output_dict['detection_boxes'])

    #1: person; 3: car; 4: motorcycle; 6: bus; 8: truck
    detection_classes = (1, 3, 4, 6, 8)
    for i in range(boxes.shape[0]):
        if (output_dict['detection_classes'][i] in detection_classes 
             and output_dict['detection_scores'][i] > threshold): 
             locs.append(boxes[i])

    # print(locs)

    flag = False
    while not flag:
        if_modify = False
        locs_num = len(locs)
        for i in range(len(locs)-1):
            x_min, y_min, x_max, y_max = locs[i]
            for j in range(i+1,len(locs)):
                # if i == j:
                #     continue
                x_min2, y_min2, x_max2, y_max2 = locs[j]
                # if (not is_small_box(locs[i],w,h)) & (not is_small_box(locs[j],w,h)):
                #     continue

                #same
                almost_same_thre = 0.01
                if (abs(x_min-x_min2)<almost_same_thre*w) and (abs(x_max-x_max2)<almost_same_thre*w) and(abs(y_max-y_max2)<almost_same_thre*h) and (abs(y_min-y_min2)<almost_same_thre*h):
                    locs[i] = [min(x_min, x_min2),min(y_min, y_min2),max(x_max, x_max2),max(y_max, y_max2)]
                    del locs[j]
                    if_modify = True
                    break

                #near
                if (is_small_box(locs[i],w,h)) or ( is_small_box(locs[j],w,h)):
                    y_near = abs((y_min+y_max)/2-(y_min2+y_max2)/2) < thre*h + (y_max-y_min)/2+(y_max2-y_min2)/2
                    x_near = abs((x_min+x_max)/2-(x_min2+x_max2)/2) < thre*w + (x_max-x_min)/2+(x_max2-x_min2)/2
                    if y_near&x_near:
                        locs[i] = [min(x_min, x_min2),min(y_min, y_min2),max(x_max, x_max2),max(y_max, y_max2)]
                        del locs[j]
                        if_modify = True
                        break

                #include
                extend = 0.1
                if(x_min<x_min2+extend*w and y_min<y_min2+extend*h and x_max>x_max2-extend*w and y_max>y_max2-extend*h) or (x_min>x_min2-extend*w and y_min>y_min2-extend*h and x_max<x_max2+extend*w and y_max<y_max2+extend*h):
                    locs[i] = [min(x_min, x_min2),min(y_min, y_min2),max(x_max, x_max2),max(y_max, y_max2)]
                    del locs[j]
                    if_modify = True
                    break


            if if_modify:
                break
        if not if_modify:
            flag = True
    # locs = np.array(locs)
    # print(locs)
    return np.asarray(locs)

def merge_boxes_and_clip(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    im_height, im_length = image_np.shape[0], image_np.shape[1]
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np, threshold)
    merged_boxes = merge_bound_box_near(output_dict,  im_height,  im_length)
    clip_image_from_merged_boxes( image_np, merged_boxes)




In [473]:
def save_image(cropped_image, image_part):
    path = './data/output/'
    Image.fromarray(cropped_image).save(path + "img" + str(image_id)+'_'+str(image_part)+ ".jpg")


In [474]:
for image_path in TEST_IMAGE_PATHS:
  image_id += 1
  show_inference(detection_model, image_path)
  merge_boxes_and_clip(detection_model, image_path)

  # # the array based representation of the image will be used later in order to prepare the
  # # result image with boxes and labels on it.
  # image_np = np.array(Image.open(image_path))
  # # Actual detection.
  # output_dict = run_inference_for_single_image(detection_model, image_np, threshold)

  # clip_image_from_box(image_np, output_dict, threshold)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


AttributeError: 'AutoTrackable' object has no attribute 'output_shapes'